In [1]:
import pandas as pd
import re
from os import system, listdir
from random import shuffle
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from random import shuffle


directory = './aclImdb/train/neg'
neg = []

for filename in os.listdir(directory):
        f = open(f'{directory}/{filename}', encoding='utf-8')
        lines = f.read()
        neg.append(lines)

directory = './aclImdb/test/pos'
pos = []

for filename in os.listdir(directory):
        f = open(f'{directory}/{filename}', encoding='utf-8')
        lines = f.read()
        pos.append(lines)

directory = './aclImdb/test/neg'


for filename in os.listdir(directory):
        f = open(f'{directory}/{filename}', encoding='utf-8')
        lines = f.read()
        neg.append(lines)

directory = './aclImdb/train/pos'
for filename in os.listdir(directory):
        f = open(f'{directory}/{filename}', encoding='utf-8')
        lines = f.read()
        pos.append(lines)


negative = pd.DataFrame(neg)
negative["label"] = 0
positive = pd.DataFrame(pos)
positive["label"] = 1
all = pd.concat([negative, positive])
all.fillna(0, inplace=True)
all = all.rename(columns={0: "text"}).reset_index(drop=True)
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
all['text'] = all['text'].apply(remove_punctuations)
all = all.sample(frac=1).reset_index(drop=True)
all

,text,label
0,When you typically watch a short film your alw...,1
1,Honestly this is one of the BEST horror movies...,1
2,Zero Day leads you to think even rethink why t...,1
3,Were it not for the fact that this came as a 2...,0
4,Favela Rising is a documentary about the slums...,1
...,...,...
49995,What a truly moronic movie all I can say is th...,0
49996,This was a pretty good movie that was overall ...,1
49997,The Hindi remake of Mrs Doubtfire starring and...,0
49998,I chose this movie by the cover which was a ba...,0


In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words='english')
vectorizer.fit(all['text'].values)
vectorized = vectorizer.transform(all['text'].values)

idf_transformer = TfidfTransformer()
idf_transformer.fit(vectorized)
listofwords = vectorizer.get_feature_names()
transformed = idf_transformer.transform(vectorized)

In [4]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import numpy as np

y = all['label'].values

X_train, X_test, y_train, y_test = train_test_split(transformed, y)

clf = SGDClassifier()
clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)
prediction = clf.predict(X_test)
print(train_score, test_score)
comparison = zip(prediction,y_test)
comp_df = pd.DataFrame(comparison)
comp_df.head(50)


0.9038666666666667 0.7332


,0,1
0,1,1
1,1,1
2,1,1
3,1,1
4,0,0
5,1,0
6,0,0
7,1,1
8,1,1
9,1,0


In [5]:
print(len(listofwords))

3121022


In [6]:
new_comment = """In all my series and movie watching years I have never seen a serie so well thought through as this. Don't let the language barrier stop you from watching this. Acting and character building are superb. Hint: grab a pen and some paper and keep track ;) Also there is totally no comparison to Stranger Things (and I have seen all 3 seasons) This is in a league of it's own entirely. Well done! Netflix more of this please!!!"""

In [7]:
new_df = pd.DataFrame([new_comment])
new_df[0] = new_df[0].apply(remove_punctuations)

In [8]:
new_df = new_df.rename(columns={0:"text"})
new_df

,text
0,In all my series and movie watching years I ha...


In [9]:

X_train_unigram = vectorizer.transform(new_df["text"].values)

X_train_unigram_tf_idf = idf_transformer.transform(X_train_unigram)

new_prediction = clf.predict(X_train_unigram_tf_idf)

In [10]:
if new_prediction[0] == 1:
    text_prediction = "Positive"
else:
    text_prediction = "Negative"
print(text_prediction)

Positive
